In [1]:
!git lfs install

Updated git hooks.
Git LFS initialized.


In [2]:
!git clone https://huggingface.co/NepBERTa/NepBERTa

Cloning into 'NepBERTa'...


In [1]:
from transformers import BertModel
model = BertModel.from_pretrained('./NepBERTa',from_tf=True)

All TF 2.0 model weights were used when initializing BertModel.

All the weights of BertModel were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertModel for predictions without further training.


In [2]:
vocab_file_dir = './NepBERTa/' 

In [3]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification

In [4]:
tokenizer = BertTokenizer.from_pretrained(vocab_file_dir,
                                        strip_accents=False,
                                         clean_text=False )

In [5]:


def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 32,
                        truncation=True,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )




In [6]:
model_weights = torch.load('./nep_berta.pt',map_location=torch.device('cpu'))

In [7]:
# Load the BertForSequenceClassification model
model = BertForSequenceClassification.from_pretrained(
    vocab_file_dir,
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
    from_tf= True
)

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5. See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(model.parameters(), 
                              lr = 5e-5,
                              eps = 1e-08
                              )

# Run on GPU


All TF 2.0 model weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.


In [8]:
model.load_state_dict(model_weights)

<All keys matched successfully>

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
import numpy as np

In [13]:
new_sentence = 'धोति रहेछ जस्तो छ यो वेले  '

# We need Token IDs and Attention Mask for inference on the new sentence
test_ids = []
test_attention_mask = []

# Apply the tokenizer
encoding = preprocessing(new_sentence, tokenizer)

# Extract IDs and Attention Mask
test_ids.append(encoding['input_ids'])
test_attention_mask.append(encoding['attention_mask'])
test_ids = torch.cat(test_ids, dim = 0)
test_attention_mask = torch.cat(test_attention_mask, dim = 0)

# Forward pass, calculate logit predictions
with torch.no_grad():
  output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

prediction = 'Hate' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'Not Hate'

print('Input Sentence: ', new_sentence)
print('Predicted Class: ', prediction)

Input Sentence:  धोति रहेछ जस्तो छ यो वेले  
Predicted Class:  Hate


In [15]:
test_ids


tensor([[    0,  8592,  1877,  4798,  3541,   355,  2001, 20330,  1016,     2,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1]])